# (a)

Use $(k)$ instead of $(i)$ for indexing data point to avoid confusion later, and use $i$ and $j$ to index the dimensions of $x$. Suppose the $x^{(k)}$ is a $n$-dimension vector.

\begin{align*}
J(\theta) &= \frac{1}{m} \sum_{k=1}^{m} \mathrm{log}(1 + e ^{-y^{(k)} \theta^T x^{(i)})} \\
               &= - \frac{1}{m} \sum_{k=1}^{m} \mathrm{log}(g(z))
\end{align*}

where $g(z) = \frac{1}{1 + e^{-z}}$ and $z = y^{(i)} \theta^T x^{(i)}$

Note that
\begin{align*}
\frac{\partial g}{\partial z} =\frac{\partial \frac{1}{1 + e^{-z}}}{\partial z}  
                              =\frac{0 - 1 \times \frac{\partial (1 + e^{-z})}{\partial z}}{(1 + e^{-z})^2} 
                              =\frac{e^{-z}}{(1 + e^{-z})^2} 
                              =\frac{1}{1 + e^{-z}} \cdot \frac{e^{-z}}{1 + e^{-z}}
                              =g(1 - g)
\end{align*}

and similarly (unnecessary for this problem, just put here for reference),

\begin{align*}
\frac{\partial (1 - g)}{\partial z} 
= \frac{\partial \frac{e^{-z}}{1 + e^{-z}}}{\partial z}
= - \frac{\partial g}{\partial z} = g(g - 1)
\end{align*}

So

\begin{align*}
\frac{\partial J}{\partial \theta_i} 
 &= -\frac{1}{m} \sum_{k=1}^{m} \frac{1}{g(z)} \frac{\partial g(z)}{\partial z} \frac{\partial z(\theta_i)}{\partial \theta_i}  \\
 &= -\frac{1}{m} \sum_{k=1}^{m} \frac{1}{g(z)} g(z) (1 - g(z)) \frac{\partial z}{\partial \theta_i}  \\
 &= -\frac{1}{m} \sum_{k=1}^{m} (1 - g(z)) \frac{\partial z}{\partial \theta_i}  \\
 &= -\frac{1}{m} \sum_{k=1}^{m} (1 - g(z)) y^{(k)} x_i^{(k)}  \\
 &= \frac{1}{m} \sum_{k=1}^{m} (g(z) - 1) y^{(k)} x_i^{(k)}  \\
\end{align*}


Then,

\begin{align*}
H_{ij} = \frac{\partial J}{\partial \theta_i \theta_j} 
&= \frac{1}{m} \sum_{k=1}^{m} \frac{\partial (g(z)  - 1)}{\partial \theta_j} y^{(k)} x_i^{(k)} \\
&= \frac{1}{m} \sum_{k=1}^{m} \frac {\partial g(z)}{\partial \theta_j} y^{(k)} x_i^{(k)} \\
&= \frac{1}{m} \sum_{k=1}^{m} g(z)(1 - g(z)) \frac{\partial z}{\partial \theta_j} y^{(k)} x_i^{(k)} \\
&= \frac{1}{m} \sum_{k=1}^{m} g(z)(1 - g(z)) y^{(k)} x_j^{(k)} y^{(k)} x_i^{(k)} \\
&= \frac{1}{m} \sum_{k=1}^{m} g(z)(1 - g(z)) x_i^{(k)} x_j^{(k)} \\
\end{align*}

Lastly,
\begin{align*}
z^THz &= \sum_{i=1}^{m} \sum_{j=1}^{m} z_i H_{ij} z_j \\
           &= \sum_{i=1}^{n} \sum_{j=1}^{n} [z_i \frac{1}{m} \sum_{k=1}^{m} g(z)(1 - g(z)) x_i^{(k)} x_j^{(k)}] z_j \\
           &= \frac{1}{m} \sum_{k=1}^{m} g(z)(1 - g(z)) \sum_{i=1}^{n} \sum_{j=1}^{n} z_i x_i^{(k)} x_j^{(k)} z_j
\end{align*}

Given that $1 > g(z) > 0$ and $\sum_{i=1}^{n} \sum_{j=1}^{n} z_i x_i^{(k)} x_j^{(k)} z_j = ((x^{(k)})^T z)^2 \ge 0$, 

$z^T H z \ge 0$, so $H$ is PSD, i.e. $H \succeq 0$. 

Hence, $J$ is convex and it has no local minima.